In [1]:
# Скачиваем необходимые библиотеки
!pip3 install qdrant-client
!pip3 install langchain
!pip3 install sentence-transformers

In [2]:
from qdrant_client import QdrantClient
from qdrant_client.http import models

In [3]:
# Запускаем Qdrant, создаем коллекцию
client = QdrantClient(':memory:')
client.create_collection(
        collection_name='my_collection',
        vectors_config=models.VectorParams(
            size=1024,
            distance=models.Distance.COSINE
        )

)

True

In [4]:
# Подготавливаем langchain
from langchain_community.vectorstores import Qdrant
from langchain_community.embeddings import HuggingFaceEmbeddings

In [5]:
# Загружаем модельку
embeddings = HuggingFaceEmbeddings(
    model_name='intfloat/multilingual-e5-large',
    model_kwargs={'device': 'cuda'}
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
import re
import numpy as np
# Читаем текст из нашей книжки (использую книгу рассказ, как и во втором задании)
text = ""
with open('drive/MyDrive/LLM_RAG/text.txt', 'r') as f:
  lines = [line.strip() for line in f.readlines()]
  text = ' '.join(lines)
preprocessed_text = re.sub(r'[.?!]+', '.', text)
len(preprocessed_text)

# Разобьем на сегменты (в сегменте по 3 предложению)
sentences = preprocessed_text.split('.')
segments = []
i = 0
while i < len(sentences):
  segments.append(' '.join(sentences[i:i+3]))
  i += 3
len(segments), len(sentences), max([len(segment) for segment in segments])

(290, 870, 983)

In [8]:
# Строим эмбеддинги
input_texts = ['passage: ' + segment for segment in segments]
embedded_values = embeddings.embed_documents(input_texts)

In [9]:
# Закидываем эмбеддинги в нашу векторную БД
client.upsert(
    collection_name='my_collection',
    points=models.Batch(
        payloads=[{'sentence': input_texts[i][9:]} for i in range(len(input_texts))],
        ids=[i for i  in range(len(input_texts))],
        vectors=embedded_values,
    ),
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [10]:
# Создаем langchain обертку для Qdrant и retriever
doc_store = Qdrant(client, collection_name='my_collection', embeddings=embeddings, content_payload_key='sentence')
retriever = doc_store.as_retriever(search_type='similarity', search_kwargs={'k': 10})

In [11]:
# Реализуем функцию поиска релевантных документов для входных запросов
def get_relevant(retriever, queries):
  return [retriever.get_relevant_documents('query: ' + query) for query in queries]
relevant_docs = get_relevant(retriever, ['какие дома существуют в столице', 'что местные думают о приезжих'])

In [12]:
# Релевантные предложения для запроса 'какие дома существуют в столице'
for i, rdoc in enumerate(relevant_docs[0]):
  print(f'{i}) {rdoc.page_content}')

0)  Теплого и комфортного помещения  Но в столице существует много прекрасных домов с дешевыми и весьма комфортными квартирами, с проведенной невской водой и с освещенной газом лестницей, при которой заводится нередко от хозяев швейцар  Обращаем еще внимание наших читателей и на самое варварство обращения с домашними животными: заезжему крокодилу, разумеется, трудно переварить подобную массу разом, и теперь он лежит, раздутый горой, и в нестерпимых страданиях ожидает смерти
1)  Первая попавшаяся мне была — «Листок», газетка без всякого особого направления, а так только вообще гуманная, за что ее преимущественно у нас презирали, хотя и прочитывали  Не без удивления прочел я в ней следующее: «Вчера в нашей обширной и украшенной великолепными зданиями столице распространились чрезвычайные слухи  Некто N
2)  В Европе давно уже преследуют судом обращающихся негуманно с домашними животными  Но, несмотря на европейское освещение, на европейские тротуары, на европейскую постройку домов, нам ещ

In [13]:
# Релевантные предложения для запроса 'что местные думают о приезжих'
for i, rdoc in enumerate(relevant_docs[1]):
  print(f'{i}) {rdoc.page_content}')

0)  Не знаем, как и объяснить подобные варварские факты, свидетельствующие о нашей незрелости и марающие нас в глазах иностранцев  Размашистость русской натуры нашла себе достойное применение  Спрашивается, чего хотелось непрошенному посетителю
1)  Французы, наехавшие с Лессепсом, предпочитают лапы, испеченные в горячей золе, что делают, впрочем, в пику англичанам, которые над ними смеются  Вероятно, у нас оценят то и другое  С своей стороны, мы рады новой отрасли промышленности, которой по преимуществу недостает нашему сильному и разнообразному отечеству
2)  Собираясь в Европу, не худо познакомиться еще на месте с населяющими ее туземцами», — и с сими словами, приняв под ручку свою супругу, тотчас же отправился с нею в Пассаж  Я же, по обыкновению моему, увязался с ними рядом — в виде домашнего друга  Никогда еще я не видел Ивана Матвеича в более приятном расположении духа, как в то памятное для меня утро, — подлинно, что мы не знаем заранее судьбы своей
3)  Теперь публикум будет ошен